#SETUP

## **Installation**
First we need to install the package. Feel free to copy the terminal command for local installation.

In [ ]:
!pip3 install --force-reinstall scrambledConvolution@git+https://github.com/tillvogt/scrambledConvolution.git

##**Imports**
next to the scrambledConvolution package, numpy and numba are installed as dependencies.
For visualizing things we import matplotlib.pyplot.

The scrambledConvolution Package can be divided in different Subpackages:


*   .layers: the different processinglayers like *'Convolutional'*, *'Dense'*, *'Pooling'* and *'Reshape'*.
*   .activations: the different activation Layers like *'Softmax'*, *'Sigmoid'*, *'Tanh'*
*   .losses: Lossfunctions like *'RMS'* and *'binary_cross_entropy'*
*   .datasets: small sample packages like a shrinked version of MNIST





In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scrambledConvolution.layers import Convolutional
from scrambledConvolution.datasets.mnist_loader import load_data

#CONVOLUTIONAL LAYER

##**Loading testdata**

Due to the fact, that this whole project is mainly about the Convolutional Layer, we try out some of the corresponding features.

So lets import some handwritten digits from MNIST Dataset. We use the build in Dataloader for that.

In [ ]:
x_train, y_train, x_test, y_test = load_data()

#change the picture_index for choosing another example
picture_index = 1
test_image = x_train[picture_index]

plt.imshow(test_image[0], cmap="gray")

##**Using 'Convolution'**

Now we instantiate a Convolutional layer. For feeding it with our testdata, just give it as argument, for the *.forward* method.

In [ ]:
conv1 = Convolutional((1,28,28), 3, 1, type="regular", mix_factor = 0)
conv_image = conv1.forward(test_image)

plt.imshow(conv_image[0], cmap="gray")

As you might see, the image is reduced in size, as well as noisy. The noise comes from the randomly picked biases, when initialising the Layer. So for investigate the kernel variations, we should set them to zero:

In [ ]:
conv1.biases = np.zeros(conv1.biases.shape)
conv_image = conv1.forward(test_image)

plt.imshow(conv_image[0], cmap="gray")

##Mix IT UP!##

Now after removing noise, let's test the mixing capabilities.
Here you can pick one of the different types:
*   "*regular*" = standard cross-correlation
*   "*retinaMix*" = the kernel stays in square shape, directs the result of operations to random positions in the resulting image
*   "*kernelMix*" = in this mode, the kernel is variated. For some of the Operations pixel inside the kernel is redirected to a random position.


All the mixing operations depend on the *mix_factor*, wich is a frequency depending on the imagesize.

In [ ]:
#setting a mixingfactor
mix_factor = 0.5

#Instantiate two Concolutional layers
conv_retina_mix = Convolutional((1,28,28), 3, 1, type="retinaMix", mix_factor=mix_factor)
conv_kernel_mix = Convolutional((1,28,28), 3, 1, type="kernelMix", mix_factor=mix_factor)
conv_conc_mix = Convolutional((1,28,28), 3, 1, type="concMix", mix_factor=mix_factor)

#Setting same weights for better comparison
conv_kernel_mix.kernels = conv_retina_mix.kernels
conv_conc_mix.kernels = conv_retina_mix.kernels

#Setting biases to zero
conv_retina_mix.biases = np.zeros(conv_retina_mix.biases.shape)
conv_kernel_mix.biases = np.zeros(conv_kernel_mix.biases.shape)
conv_conc_mix.biases = np.zeros(conv_conc_mix.biases.shape)

#feeding it with the testimage
retina_mix_image = conv_retina_mix.forward(test_image)
kernel_mix_image = conv_kernel_mix.forward(test_image)
conc_mix_image = conv_conc_mix.forward(test_image)

######plotting#####
fig, axs = plt.subplots(1, 3, figsize=(10, 5))

axs[0].imshow(retina_mix_image[0], cmap="gray")
axs[0].title.set_text(f"Retinamix: {mix_factor}")
axs[0].axis("off")

axs[1].imshow(kernel_mix_image[0], cmap="gray")
axs[1].title.set_text(f"Kernelmix: {mix_factor}")
axs[1].axis("off")

axs[2].imshow(conc_mix_image[0], cmap="gray")
axs[2].title.set_text(f"Concentrationmix: {mix_factor}")
axs[2].axis("off")

plt.tight_layout()
plt.show()

#TRAIN

##Essential imports

For training a Network, one has to make some further imports first.

In [ ]:
from scrambledConvolution import train
from scrambledConvolution.losses import mse, mse_prime
from scrambledConvolution.activations import Sigmoid, Softmax
from scrambledConvolution.layers import *

##Building Network

Now we are able to build a Network as a list of the different layer Classes.

In [ ]:
convolution_type = "kernelMix"
mix_factor = 0


network = [
      Convolutional((1, 28, 28), 5, 5, type=convolution_type, mix_factor=mix_factor),
      Sigmoid(),
      Pooling(),
      Convolutional((5, 12, 12), 5, 5, type=convolution_type, mix_factor=mix_factor),
      Sigmoid(),
      Pooling(),
      Reshape((5,4,4), ((5*4*4), 1)),
      Dense(80, 32),
      Sigmoid(),
      Dense(32,10),
      Softmax()
]

##Do you even lift?

Fortunatly the pc trains for us...

Just use the *train()* function and set the essential parameters. A fews things to explain:

*   *friction:* There is no real optimizier implemented. However, to overcome saddle points 'inertia' is implemented. the *friction* parameter is the factor of the previous gradient. I recomment a value between 0 (no inertia) to 1.
*   *weight-saving:* If this is TRUE, the weights and biases of convolutional Layers are saved in a .csv at the end of the training

In [ ]:
_ = train(mix_factor, network, mse, mse_prime, x_train, y_train, x_test, y_test, epochs = 10, batch_size = 16, )